In [30]:
from google.colab import files
import pandas as pd

# Upload file from local system
uploaded = files.upload()

Saving Transactions.csv to Transactions (1).csv
Saving Customers.csv to Customers (1).csv
Saving Products.csv to Products (1).csv


In [31]:
# Now I need to load each CSV file into a pandas dataframe
transactions_df = pd.read_csv('/content/Transactions.csv')
customers_df = pd.read_csv('/content/Customers.csv')
products_df = pd.read_csv('/content/Products.csv')


In [32]:
# Displaying the first few rows of each dataframe
print("Transactions DataFrame:")
print(transactions_df.head())

print("\nCustomers DataFrame:")
print(customers_df.head())

print("\nProducts DataFrame:")
print(products_df.head())

Transactions DataFrame:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue   Price  
0      300.68  300.68  
1      300.68  300.68  
2      300.68  300.68  
3      601.36  300.68  
4      902.04  300.68  

Customers DataFrame:
  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15

Products DataFrame:
  

In [33]:
# Importing necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
# Preprocessing: Merge datasets for analysis purpose
merged_df = transactions_df.merge(customers_df, on='CustomerID').merge(products_df, on='ProductID')


In [35]:
# Displaying the first few rows of each dataset to analyze their structure
transactions_head = transactions_df.head()
customers_head = customers_df.head()
products_head = products_df.head()

In [36]:
# Here I need to get basic information about each dataset to work further in detail
transactions_info = transactions_df.info()
customers_info = customers_df.info()
products_info = products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price            1000 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 54.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
<class 'pandas.core.fr

In [37]:

# Check if there are any missing values in the datasets
transactions_missing = transactions_df.isnull().sum()
customers_missing = customers_df.isnull().sum()
products_missing = products_df.isnull().sum()

In [38]:
transactions_head, customers_head, products_head, transactions_missing, customers_missing, products_missing

(  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
 0        T00001      C0199      P067  2024-08-25 12:38:23         1   
 1        T00112      C0146      P067  2024-05-27 22:23:54         1   
 2        T00166      C0127      P067  2024-04-25 07:38:55         1   
 3        T00272      C0087      P067  2024-03-26 22:55:37         2   
 4        T00363      C0070      P067  2024-03-21 15:10:10         3   
 
    TotalValue   Price  
 0      300.68  300.68  
 1      300.68  300.68  
 2      300.68  300.68  
 3      601.36  300.68  
 4      902.04  300.68  ,
   CustomerID        CustomerName         Region  SignupDate
 0      C0001    Lawrence Carroll  South America  2022-07-10
 1      C0002      Elizabeth Lutz           Asia  2022-02-13
 2      C0003      Michael Rivera  South America  2024-03-07
 3      C0004  Kathleen Rodriguez  South America  2022-10-09
 4      C0005         Laura Weber           Asia  2022-08-15,
   ProductID              ProductName     Catego

In [39]:
# Preprocessing: Merge datasets for analysis
merged_df = transactions_df.merge(customers_df, on='CustomerID').merge(products_df, on='ProductID')


In [40]:
# Aggregating data by CustomerID
customer_features = merged_df.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'ProductID': 'nunique'  # Number of unique products purchased
}).reset_index()


In [41]:
# Handling potential missing values
customer_features.fillna(0, inplace=True)

# Setting CustomerID as index for similarity calculation
customer_features.set_index('CustomerID', inplace=True)

# Here now I will Compute similarity matrix
similarity_matrix = cosine_similarity(customer_features)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features.index, columns=customer_features.index)


In [42]:
# This function will be used to get top 3 lookalike customers
def get_top_lookalikes(customer_id, top_n=3):
    if customer_id in similarity_df.index:
        similar_customers = similarity_df[customer_id].sort_values(ascending=False)[1:top_n+1]
        return list(zip(similar_customers.index, similar_customers.values))
    else:
        return []

In [43]:
# Finally I generated Lookalike.csv for first 20 customers
lookalike_results = {}
for customer_id in customers_df['CustomerID'].head(20):
    lookalike_results[customer_id] = get_top_lookalikes(customer_id)

lookalike_df = pd.DataFrame.from_dict(lookalike_results, orient='index', columns=['Lookalike1', 'Lookalike2', 'Lookalike3'])
lookalike_df.to_csv('Aditya_Kumar_Pandey_Lookalike.csv')

print("Lookalike model results saved as NithinReddy_Lookalike.csv")

Lookalike model results saved as Aditya_Kumar_Pandey_Lookalike.csv
